# IBM Applied Data Science Capstone Final Project

### Week 5 Final Proejct Python File (Code) : Opening a New Shopping Mall in Mumbai, India

1. Build a dataframe of neighborhoods in Mumbai, India by web scraping the data from Wikipedia page
2. Get the geographical coordinates of the neighborhoods
3. Obtain the venue data for the neighborhoods from Foursquare API
4. Explore and cluster the neighborhoods
5. Select the best cluster to open a new shopping mall

#### Import libraries

In [4]:
pip install geocoder

     |████████████████████████████████| 98 kB 9.6 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install folium

     |████████████████████████████████| 94 kB 6.3 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values # to get coordinates

import requests


import geocoder# library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries  are imported.")

Libraries  are imported.


#### Scrap data from Wikipedia page into a DataFrame

In [8]:
# send the GET request
data = requests.get("https://commons.wikimedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [9]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [10]:
# create a list to store neighborhood data
neighborhoodList = []

In [11]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [12]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
kl_df.head()

,Neighborhood
0,"► Aarey Milk Colony‎ (1 C, 29 F)"
1,"► Andheri‎ (6 C, 14 F)"
2,"► Anushakti Nagar‎ (1 C, 6 F)"
3,"► Bandra‎ (12 C, 99 F)"
4,"► Borivali‎ (3 C, 18 F)"


In [13]:
kl_df.shape

(31, 1)

#### Get the geographical coordinates

In [14]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [15]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [16]:
coords

[[19.17026753178864, 72.87107204158183],
 [19.103930000000048, 72.86698000000007],
 [19.042830000000038, 72.92734000000007],
 [18.936010002594443, 72.82443998978681],
 [19.22568503498364, 72.85729009325489],
 [19.20425998941262, 72.87849999632616],
 [19.06887000000006, 72.90690000000006],
 [19.085449990390543, 72.90928999563181],
 [19.161530000000027, 72.85618000000005],
 [19.06117000000006, 72.91689000000008],
 [19.14116000000007, 72.84332000000006],
 [19.11497000563655, 72.83018000090038],
 [18.92724000000004, 72.83279000000005],
 [18.96172000000007, 72.82625000000007],
 [19.20694000000003, 72.83494000000007],
 [19.131380000000036, 72.93568000000005],
 [18.946860000000072, 72.82889000000006],
 [19.179380000000037, 72.81667000000004],
 [19.119050000000072, 72.88281000000006],
 [18.968360000000075, 72.84174000000007],
 [19.26570462175641, 72.8706928508165],
 [19.26134645237399, 72.87328667016129],
 [19.174370000000067, 72.95068000000003],
 [18.935840017598053, 72.82634997406339],
 [19.

In [17]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [18]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [19]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(31, 3)


,Neighborhood,Latitude,Longitude
0,"► Aarey Milk Colony‎ (1 C, 29 F)",19.170268,72.871072
1,"► Andheri‎ (6 C, 14 F)",19.103930,72.866980
2,"► Anushakti Nagar‎ (1 C, 6 F)",19.042830,72.927340
3,"► Bandra‎ (12 C, 99 F)",18.936010,72.824440
4,"► Borivali‎ (3 C, 18 F)",19.225685,72.857290
5,"► Charni Road‎ (4 C, 1 F)",19.204260,72.878500
6,"► Chembur‎ (1 C, 38 F)",19.068870,72.906900
7,"► Ghatkopar‎ (2 C, 8 F)",19.085450,72.909290
8,"► Goregaon‎ (1 C, 9 F)",19.161530,72.856180
9,"► Govandi‎ (1 C, 3 F)",19.061170,72.916890


In [20]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

#### Create a map of Bangalore with neighborhoods superimposed on top

In [23]:
# get the coordinates of Mumbai
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-applicationDS")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 19.0759899, 72.8773928.


In [24]:
# create map of Mumbai using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [25]:
# save the map as HTML file
map_kl.save('map_kl.html')

#### Use the Foursquare API to explore the neighborhoods

In [26]:
CLIENT_ID = 'RZTE3IWG3UZCE3DSGBYBG54WTURKAMZTM5VG2KHT0CHO5JRZ'
CLIENT_SECRET = '3GL4AP33KZQUCVZVCEIZQZXH2NT5LRDVSCM3TBGF02MF2DFV'
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RZTE3IWG3UZCE3DSGBYBG54WTURKAMZTM5VG2KHT0CHO5JRZ
CLIENT_SECRET:3GL4AP33KZQUCVZVCEIZQZXH2NT5LRDVSCM3TBGF02MF2DFV


##### Now, let's get the top 50 venues that are within a radius of 2000 meters.

In [27]:
radius = 2000
LIMIT = 50

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [28]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1328, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"► Aarey Milk Colony‎ (1 C, 29 F)",19.170268,72.871072,The Westin Mumbai Garden City,19.172654,72.860518,Hotel
1,"► Aarey Milk Colony‎ (1 C, 29 F)",19.170268,72.871072,PVR Cinemas,19.174016,72.860485,Multiplex
2,"► Aarey Milk Colony‎ (1 C, 29 F)",19.170268,72.871072,Starbucks,19.174177,72.860350,Coffee Shop
3,"► Aarey Milk Colony‎ (1 C, 29 F)",19.170268,72.871072,Eighteen Lounge and Bar,19.172648,72.860513,Hotel Bar
4,"► Aarey Milk Colony‎ (1 C, 29 F)",19.170268,72.871072,British Brewing Company (BBC),19.174186,72.860504,Brewery


##### Let's check how many venues were returned for each neighorhood

In [29]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"► Aarey Milk Colony‎ (1 C, 29 F)",42,42,42,42,42,42
"► Andheri‎ (6 C, 14 F)",50,50,50,50,50,50
"► Anushakti Nagar‎ (1 C, 6 F)",17,17,17,17,17,17
"► Bandra‎ (12 C, 99 F)",50,50,50,50,50,50
"► Borivali‎ (3 C, 18 F)",50,50,50,50,50,50
"► Charni Road‎ (4 C, 1 F)",50,50,50,50,50,50
"► Chembur‎ (1 C, 38 F)",50,50,50,50,50,50
"► Ghatkopar‎ (2 C, 8 F)",50,50,50,50,50,50
"► Goregaon‎ (1 C, 9 F)",50,50,50,50,50,50


##### Let's find out how many unique categories can be curated from all the returned venues

In [30]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 151 uniques categories.


In [31]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Hotel', 'Multiplex', 'Coffee Shop', 'Hotel Bar', 'Brewery',
       'Donut Shop', 'Shopping Mall', 'Asian Restaurant',
       'Breakfast Spot', 'Sandwich Place', 'Bakery', 'Indian Restaurant',
       'Clothing Store', 'Bar', 'Vegetarian / Vegan Restaurant', 'Café',
       'Fast Food Restaurant', 'Gym', 'Bookstore', 'Bistro',
       'Italian Restaurant', 'Event Space', 'Electronics Store',
       'Pizza Place', 'Farm', 'Office', 'Seafood Restaurant',
       'Food Court', 'Restaurant', 'Lounge', 'Airport Service', 'Spa',
       'Bagel Shop', 'Airport Lounge', 'Nightclub', 'Falafel Restaurant',
       'Beer Garden', 'Chinese Restaurant', 'Juice Bar', 'Snack Place',
       'Airport', 'Food', 'Pub', 'Plaza', 'Bus Station',
       'Paper / Office Supplies Store', 'Music Venue', 'Hot Dog Joint',
       'Food Truck', 'Tunnel'], dtype=object)

In [32]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

#### Analyze Neighborhood

In [33]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(1328, 152)


,Neighborhoods,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,Gift Shop,Goan Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Lounge,Maharashtrian Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Mountain,Movie Theater,Mughlai Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pizza Place,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Women's Store,Zoo
0,"► Aarey Milk Colony‎ (1 C, 29 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"► Aarey Milk Colony‎ (1 C, 29 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"► Aarey Milk Colony‎ (1 C, 29 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"► Aarey Milk Colony‎ (1 C, 29 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"► Aarey Milk Colony‎ (1 C, 29 F)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(31, 152)


,Neighborhoods,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,Gift Shop,Goan Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Lounge,Maharashtrian Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Mountain,Movie Theater,Mughlai Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pizza Place,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Women's Store,Zoo
0,"► Aarey Milk Colony‎ (1 C, 29 F)",0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.023810,0.00,0.000000,0.00,0.023810,0.047619,0.00,0.00,0.000000,0.00,0.000000,0.02381,0.02381,0.00,0.00,0.00,0.02381,0.02381,0.00,0.000000,0.000000,0.023810,0.00,0.000000,0.023810,0.00,0.047619,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.047619,0.023810,0.02381,0.000000,0.02381,0.142857,0.000000,0.00000,0.00,0.000000,0.00,0.02381,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.000000,0.00,0.00,0.00,0.00,0.02381,0.00,0.00,0.00,0.00,0.00000,0.000000,0.047619,0.02381,0.000000,0.00,0.047619,0.00,0.02381,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.000000,0.00,0.047619,0.00,0.000000,0.00,0.00,0.00,0.00,0.02381,0.000000,0.000000,0.000000,0.00,0.00,0.023810,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.023810,0.00,0.02381,0.000000,0.047619,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.071429,0.00,0.00,0.000000,0.00000
1,"► Andheri‎ (6 C, 14 F)",0.00,0.02,0.02,0.02,0.00,0.000000,0.00,0.020000,0.00,0.000000,0.02,0.000000,0.000000,0.00,0.00,0.020000,0.00,0.000000,0.00000,0.00000,0.00,0.00,0.00,0.00000,0.02000,0.00,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.00,0.120000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.020000,0.00000,0.040000,0.000000,0.00000,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00000,0.00,0.00,0.00,0.00,0.00000,0.000000,0.200000,0.00000,0.000000,0.00,0.140000,0.00,0.08000,0.00,0.00,0.02,0.00,0.06,0.00,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.02,0.00,0.00000,0.000000,0

In [35]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

6

##### Create a new DataFrame for Shopping Mall data only

In [36]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [37]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,"► Aarey Milk Colony‎ (1 C, 29 F)",0.047619
1,"► Andheri‎ (6 C, 14 F)",0.000000
2,"► Anushakti Nagar‎ (1 C, 6 F)",0.000000
3,"► Bandra‎ (12 C, 99 F)",0.000000
4,"► Borivali‎ (3 C, 18 F)",0.000000


##### Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Mumbai into 4 clusters.

In [38]:
# set number of clusters
kclusters = 4

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 0, 0, 0, 0, 2, 0, 2, 2, 0], dtype=int32)

In [39]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [40]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,"► Aarey Milk Colony‎ (1 C, 29 F)",0.047619,3
1,"► Andheri‎ (6 C, 14 F)",0.000000,0
2,"► Anushakti Nagar‎ (1 C, 6 F)",0.000000,0
3,"► Bandra‎ (12 C, 99 F)",0.000000,0
4,"► Borivali‎ (3 C, 18 F)",0.000000,0


In [41]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(31, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"► Aarey Milk Colony‎ (1 C, 29 F)",0.047619,3,19.170268,72.871072
1,"► Andheri‎ (6 C, 14 F)",0.000000,0,19.103930,72.866980
2,"► Anushakti Nagar‎ (1 C, 6 F)",0.000000,0,19.042830,72.927340
3,"► Bandra‎ (12 C, 99 F)",0.000000,0,18.936010,72.824440
4,"► Borivali‎ (3 C, 18 F)",0.000000,0,19.225685,72.857290


In [42]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(31, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
30,"► Vile Parle‎ (1 C, 2 F)",0.000000,0,19.096240,72.850240
25,"► Powai‎ (4 C, 54 F)",0.000000,0,19.123120,72.909350
21,"► Mira-Bhayandar‎ (2 C, 17 F)",0.000000,0,19.261346,72.873287
20,► Mira Road‎ (5 F),0.000000,0,19.265705,72.870693
19,"► Mazagaon‎ (3 C, 11 F)",0.000000,0,18.968360,72.841740
18,► Marol‎ (4 F),0.000000,0,19.119050,72.882810
17,"► Malad‎ (4 C, 10 F)",0.000000,0,19.179380,72.816670
16,"► Mahalaxmi, Mumbai‎ (7 C, 15 F)",0.000000,0,18.946860,72.828890
29,"► Vasai-Virar‎ (5 C, 1 P, 62 F)",0.000000,0,18.936190,72.824420
14,"► Kandivali‎ (1 C, 3 F)",0.000000,0,19.206940,72.834940


##### Finally, let's visualize the resulting clusters

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [44]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

#### Examine Clusters

##### Cluster 0

In [45]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
30,"► Vile Parle‎ (1 C, 2 F)",0.0,0,19.096240,72.850240
25,"► Powai‎ (4 C, 54 F)",0.0,0,19.123120,72.909350
21,"► Mira-Bhayandar‎ (2 C, 17 F)",0.0,0,19.261346,72.873287
20,► Mira Road‎ (5 F),0.0,0,19.265705,72.870693
19,"► Mazagaon‎ (3 C, 11 F)",0.0,0,18.968360,72.841740
18,► Marol‎ (4 F),0.0,0,19.119050,72.882810
17,"► Malad‎ (4 C, 10 F)",0.0,0,19.179380,72.816670
16,"► Mahalaxmi, Mumbai‎ (7 C, 15 F)",0.0,0,18.946860,72.828890
29,"► Vasai-Virar‎ (5 C, 1 P, 62 F)",0.0,0,18.936190,72.824420
14,"► Kandivali‎ (1 C, 3 F)",0.0,0,19.206940,72.834940


##### Cluster 1

In [46]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
15,"► Kanjurmarg‎ (1 C, 51 F)",0.032258,1,19.13138,72.93568


##### Cluster 2

In [47]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
8,"► Goregaon‎ (1 C, 9 F)",0.02,2,19.16153,72.85618
7,"► Ghatkopar‎ (2 C, 8 F)",0.02,2,19.08545,72.90929
5,"► Charni Road‎ (4 C, 1 F)",0.02,2,19.20426,72.87850


##### Cluster 3

In [48]:
kl_merged.loc[kl_merged['Cluster Labels'] == 3]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
22,"► Mulund‎ (1 C, 17 F)",0.040000,3,19.174370,72.950680
0,"► Aarey Milk Colony‎ (1 C, 29 F)",0.047619,3,19.170268,72.871072


#### OBSERVATIONS:

Main focusing shopping Mall arround the Mumbai city with the highest number in cluster 3 and moderate number in cluster 1 and on cluster 1 is good compare to cluster 2 On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 3 which already have high concentration of shopping malls and suffering from intense competition.